# Verification of Stable Signature Implementation

In [1]:
key = "111010110101000001010111010011010100010000100111"

In [2]:
from PIL import Image
import torch
import torchvision.transforms as transforms

def msg2str(msg):
    return "".join([('1' if el else '0') for el in msg])

def str2msg(str):
    return [True if el=='1' else False for el in str]

msg_extractor = torch.jit.load("./models/decoder_whit.pth").to("cuda")
transform_imnet = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

In [3]:
img = Image.open("./output/imgs/000_val_w.png")
img = transform_imnet(img).unsqueeze(0).to("cuda")
msg = msg_extractor(img) # b c h w -> b k
bool_msg = (msg>0).squeeze().cpu().numpy().tolist()
print("Extracted message: ", msg2str(bool_msg))
print("Original message:  ", key)
mismatch_bits = 0
for i in range(len(bool_msg)):
    if bool_msg[i] != bool(int(key[i])):
        mismatch_bits += 1
print(f"{mismatch_bits} out of {len(bool_msg)} bits were mismatched")

Extracted message:  111010110101000001010111010011010100010000100111
Original message:   111010110101000001010111010011010100010000100111
0 out of 48 bits were mismatched
